# WebPie 

  *web services development framework for Python*
  
  https://webpie.github.io/

# Hello, World !

In [ ]:
from webpie import WPApp

def hello(request, relpath, **args):      # hello is a web method
    return "Hello, World !\n"             # always responds with "Hello, World !"

WPApp(hello).run_server(9000)             # create application and run it as HTTP server

```sh
$ curl http://localhost:9000
Hello, World !
```

# Web Method Arguments: request

In [ ]:
from webpie import WPApp

def hello(request, relpath, **args):      # hello is a web method
    return "Hello, World !\n"             # always responds with "Hello, World !"

WPApp(hello).run_server(9000)             # create application and run it as HTTP server

``request`` is a WebOb Request object
* Parsed HTTP request + WSGI environment
* See http://www.webob.org/

# Web Method Arguments: relpath

In [ ]:
from webpie import WPApp

def hello(request, relpath, **args):
    who = relpath or "World"
    return f"Hello, {who} !\n"             

WPApp(hello).run_server(9000)             

``relpath`` is the URL path relative to the path to the web method

    http://localhost:9000/webpie
        -> Path: /webpie
            -> path to web method: "/" (in our case - top)
            ->               tail: "webpie" -> relpath argument 

```sh
$ curl http://localhost:9000/            # relpath is empty
Hello, World !

$ curl http://localhost:9000/webpie      # relpath is "webpie"
Hello, webpie !
```

# Web Method Arguments: key=value

In [ ]:
from webpie import WPApp

def hello(request, relpath, name="world"):      
    return f"Hello, {name} !\n"             

WPApp(hello).run_server(9000)             

Named URL query arguments passed to the web method as keyword arguments

    http://localhost:9000/hello?name=webpie
        -> Query arguments:
            name="webpie"

```sh
$ curl http://localhost:9000/hello
Hello, world !

$ curl http://localhost:9000/hello?name=webpie
Hello, webpie !
```

# Handlers

* What if we want to have more than one web method in the application ?
* Create a subclass of the WPHandler class and add multiple methods

In [ ]:
from webpie import WPApp, WPHandler
import time

class Handler(WPHandler):
    
    def hello(self, request, relpath, name="stranger"):      
        return f"Hello, {name}\n"
    
    def time(self, request, relpath):
        return time.ctime() + "\n"

WPApp(Handler).run_server(9000)

```sh
$ curl http://localhost:8000/hello
Hello, stranger

$ curl http://localhost:8000/hello?name=world
Hello, world

$ curl http://localhost:8000/time
Fri Nov 12 15:16:07 2021
```

# Nested Handlers

* Handlers can be nested: top handler / subhandler / subhandler / ... / web method
* URL path *is* the path through the handlers/subhandlrers tree from the top to the web method
* Better, more manageable structure of the code, object oriented design

In [ ]:
from webpie import WPApp, WPHandler
import time

class Clock(WPHandler):
    def time(self, request, relpath):             # <- http://host:port/clock/time
        return time.ctime() + "\n"

class Greeter(WPHandler):
    def hello(self, request, relpath, name="stranger"):    # <- http://host:port/greet/hello 
        return f"Hello, {name}\n"

class RootHandler(WPHandler):
    def __init__(self, *params):
        WPHandler.__init__(self, *params)
        self.greet = Greeter(*params)             # <- http://host:port/greet/* 
        self.clock = Clock(*params)               # <- http://host:port/clock/*
    
WPApp(RootHandler).run_server(9000)

```sh
$ curl http://localhost:9000/clock/time
Fri Nov 12 15:30:56 2021
$ curl http://localhost:9000/greet/hello
Hello, stranger
```

# What exactly does a web method return ?

* Strictly speaking a web method is supposed to return ``WebOb Response`` object
  * full power of HTTP response building


* Convenience return options
  * str -> HTTP response with given body and 200 status code
  * bytes -> HTTP response with given body and 200 status code
  * int -> HTTP response with given status code, empty body
  * iterable -> chunked response body, 200 status code
  * (int, string) -> status code, response body
  * (int, iterable) -> status code, response body
  * (iterable, string) -> body, content type
  * (iterable, dictionary) -> body, headers=dictionary
  * dictionary -> body=json.dumps(dictionary), content_type="text/json"
  * ... *many more* - all will be converted to Response(...) by webpie
 

In [ ]:
def method(self, request, relpath, **args):
    ...
    return (format_csv(tup)+"\n" for tup in my_data), "text/csv"
    # equivalent: return Response(app_iter=(format_csv(...) ...), content_type="text/cvs")

# App and Handler Lifetime

* WPApp object is *persistent*
  * Created once when the server process starts
  * Can be used to store context across requests

* WPHandler object is *transient*
  * Created and destroyed for each request
  * Has access to the WPApp object via ``self.App``, also passed to the Handler's ``__init__``
  * Request (WebOb) object is passed to the Handler ``__init__`` and to the web method

In [ ]:
class MyHandler(WPHandler):
    
    def __init__(self, request, app):
        WPHandler.__init__(self, request, app)
        ...
        
    def method(self, request, relpath, *kw):
        ...
        app = self.App     # the persistent WPApp object

# Threaded Applications

* Built-in WebPie HTTP server as well as middlewares like uWSGI execute HTTP requests on threads
* WebPie provides several mechanisms to support inter-thread synchronization/locking
  * @app_synchronized decorator - coarse
  * WPApp object as context manager - more fine grain





# WPApp as locking context manager

In [ ]:
from webpie import WPApp, WPHandler
class Handler(WPHandler):
    
    def set(self, request, relpath, **kw):        # URL: .../set?name=value
        name, value = list(kw.items())[0]         # extract name, value
        with self.App:                            # lock the App
            self.App.Memory[name] = value         # record into App memory
            return "OK\n"
    
    def get(self, request, varname):
        with self.App:                            # lock the App
            value = self.App.Memory.get(varname)
            return value + "\n" if value is not None else 404

class App(WPApp):
    
    def __init__(self, *params, **args):
        WPApp.__init__(self, *params, **args)
        self.Memory = {}                          # accessible by concurrent threads
        
App(Handler).run_server(9000, max_connections=10) # up to 10 concurrent request threads

```sh
$ curl http://localhost:9000/get/x
# (not found)
$ curl http://localhost:9000/set?x=2
OK
$ curl http://localhost:9000/get/x
2
```

# @app_synchronized decorator

In [ ]:
    @app_synchronized
    def web_method(...):
        # code
        
    # equivalent:
    def web_method(...):
        with self.App:
            # code

In [ ]:
from webpie import WPApp, WPHandler, app_synchronized
class Handler(WPHandler):
    
    @app_synchronized
    def set(self, request, relpath, **kw):        # URL: .../set?name=value
        name, value = list(kw.items())[0]         # extract name, value
        self.App.Memory[name] = value             # record into App memory
        return "OK\n"
    
    @app_synchronized
    def get(self, request, varname):
        value = self.App.Memory.get(varname)
        return value + "\n" if value is not None else 404

class App(WPApp):
    
    def __init__(self, *params, **args):
        WPApp.__init__(self, *params, **args)
        self.Memory = {}
        
App(Handler).run_server(9000, max_connections=10) # up to 10 concurrent request threads

# WSGI

* WSGI - Web Server Gateway Interface - standard for web server development in Python
  * https://www.python.org/dev/peps/pep-3333/
  * https://docs.python.org/3/library/wsgiref.html#module-wsgiref
* Apache/mod_wsgi, Ngix/uWSGI - available web servers where a WSGI application can be plugged
* Any WPApp object *is* a WSGI application

In [ ]:
from webpie import WPApp, WPHandler
import time

class Handler(WPHandler):
    def hello(self, request, relpath, name="stranger"):      
        return f"Hello, {name}\n"
    def time(self, request, relpath):
        return time.ctime() + "\n"

application = WPApp(Handler)         # WSGI application

```sh
$ uwsgi --wsgi-file wsgi_app.py --http :9000 
```

# uWSGI

* Run same code standalone or under uWSGI

In [ ]:
from webpie import WPApp, WPHandler
import time

class Handler(WPHandler):
    def hello(self, request, relpath, name="stranger"):      
        return f"Hello, {name}\n"
    def time(self, request, relpath):
        return time.ctime() + "\n"

application = WPApp(Handler)

if __name__ == "__main__":
    # running stand-alone
    import sys
    application.run_server(int(sys.argv[1]))

```sh
$ uwsgi --wsgi-file wsgi_app.py --http :9000 
$ python wsgi_app.py 9000
```

# Other Features

* Support for sessions - use WPSessionApp instead of WPApp
  * uses cookies to carry session id
  * persistent session data storage
* Static file server - WPStaticHandler - pluggable into the Handler tree
  * Add images, icons, JavaScript, CSS, etc. to your application
* Method restrictions - strict applications
  * not all methods of Handler class need to be exposed as web methods
  * methods starting with "_" are not web methods
* HTTPS support
* Jinja2 support - if jinja2 is installed
* Built in HTTP/HTTPS server
  * multithreaded
  * multi-port
  * multi-process
  * multi-app - route by URL path prefix


# Installation

```sh
$ pip install webpie

or

$ pip install --user webpie
```